# Heart Disease Predictor

## Background

WHO estimates that 17.9 million people die from cardiovascular diseases (CVDs) every year.<br>
There are multiple factors taht could contribute to Cardivascular Diseases:<br>
- Unhealthy diet
- Lack of physical activity
- Mental illness<br>
  
We are working with a R&D company that has gathered anonymized data from multiple hospitals. The [data](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)

## Goal

Identifying these risk factors early on could help prevent many premature deaths.

## Libraries

In [1]:
import pandas as pd

## Data dictionary

* **Age**: age of the patient [years]
* **Sex**: sex of the patient [M: Male, F: Female]
* **ChestPainType**: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
* **RestingBP**: resting blood pressure [mm Hg]
* **Cholesterol**: serum cholesterol [mm/dl]
* **FastingBS**: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
* **RestingECG**: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
* **MaxHR**: maximum heart rate achieved [Numeric value between 60 and 202]
* **ExerciseAngina**: exercise-induced angina [Y: Yes, N: No]
* **Oldpeak**: oldpeak = ST [Numeric value measured in depression]
* **ST_Slope**: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
* **HeartDisease**: output class [1: heart disease, 0: Normal]

## Load Data

In [4]:
heart_df = pd.read_csv('dataset/heart.csv')

## EDA

In [5]:
heart_df.shape

(918, 12)

In [6]:
heart_df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [ ]:
# Visualizations



### Observations

- The patients have an age range between 28 and 77 years old
  - Our model will be able to confidently predict for that range
- Around 80% of patients are Male
- Around 55% of the observations have hearth disease 

## Data cleaning

### Data cleaning

The data looks pretty clean

### Missing data

In [12]:
heart_df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

No missing data

## Feature Engineering

- dummies
- feature selection
- 

## Split data


- split data into training, validation and test



## Feature Engineering II

- standardize data

## Model fitting and evaluation

## Hyperparameter optimization

## Testing model

## Exporting model